# Imports

In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
from pathlib import Path
import tempfile
import functools

import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [3]:
import roicat

# Find paths to data

##### Prepare list of paths to data

In this example we are using suite2p output files, but other data types can be used (CaImAn, etc.) \
See the notebook on ingesting diverse data: https://github.com/RichieHakim/ROICaT/blob/main/notebooks/jupyter/other/demo_custom_data_importing.ipynb

Make a list containing the paths to all the input files.

In this example we are using suite2p, so the following are defined:
1. `paths_allStat`: a list to all the stat.npy files
2. `paths_allOps`: a list with ops.npy files that correspond 1-to-1 with the stat.npy files

In [ ]:
dir_allOuterFolders = r'/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_0322R/statFiles/'

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'

paths_allStat = roicat.helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=4,
)[:]
paths_allOps  = np.array([Path(path).resolve().parent / pathSuffixToOps for path in paths_allStat])[:]

print(f'paths to all stat files:');
[print(path) for path in paths_allStat];
print('');
print(f'paths to all ops files:');
[print(path) for path in paths_allOps];


**Important parameters**:

- `um_per_pixel` (float):
    - Resolution. 'micrometers per pixel' of the imaging field of view.

In [ ]:
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    um_per_pixel=2.5,  
    new_or_old_suite2p='new',
    type_meanImg='meanImgE',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

# ROInet embedding

This step passes the images of each ROI through the ROInet neural network. The inputs are the images, the output is an array describing the visual properties of each ROI.

##### 1. Initialize ROInet

Initialize the ROInet object. The `ROInet_embedder` class will automatically download and load a pretrained ROInet model. If you have a GPU, this step will be much faster.

In [ ]:
DEVICE = roicat.helpers.set_device(use_GPU=True, verbose=True)
dir_temp = tempfile.gettempdir()

roinet = roicat.ROInet.ROInet_embedder(
    device=DEVICE,  ## Which torch device to use ('cpu', 'cuda', etc.)
    dir_networkFiles=dir_temp,  ## Directory to download the pretrained network to
    download_method='check_local_first',  ## Check to see if a model has already been downloaded to the location (will skip if hash matches)
    download_url='https://osf.io/c8m3b/download',  ## URL of the model
    download_hash='357a8d9b630ec79f3e015d0056a4c2d5',  ## Hash of the model file
    forward_pass_version='head',  ## How the data is passed through the network
    verbose=True,  ## Whether to print updates
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,  ## Input images of ROIs
    um_per_pixel=data.um_per_pixel,  ## Resolution of FOV
    pref_plot=False,  ## Whether or not to plot the ROI sizes
);

##### 2. Check ROI_images sizes
In general, you want to see that a neuron fills roughly 25-50% of the area of the image. \
**Adjust `um_per_pixel` above to rescale image size**

In [ ]:
roicat.visualization.display_toggle_image_stack(roinet.ROI_images_rs[:1000], image_size=(200,200))

##### 3. Pass data through network

Pass the data through the network. Expect for large datasets (~40,000 ROIs) that this takes around 15 minutes on CPU or 1 minute on GPU.

In [ ]:
roinet.generate_latents();

# Load classifier

Hopefully you've run the [classifier_training_interactive notebook](https://github.com/RichieHakim/ROICaT/blob/main/notebooks/jupyter/classification/classifier_train_interactive.ipynb) and have saved a .onnx file containing the classifier model.

##### load model

In [ ]:
classifier = roicat.classification.classifier.Load_ONNX_model_sklearnLogisticRegression('/home/rich/Desktop/model.onnx')

##### Run data through model

In [12]:
predictions, probabilities = classifier(roinet.latents)

results = {
    'preds': predictions,
    'probs': probabilities,
}

# Visualize results

In [ ]:
u, c = np.unique(predictions, return_counts=True)

plt.figure()
plt.bar(u, c)
plt.xlabel('predicted class')
plt.ylabel('counts')

# Save outputs

Specify save location

In [14]:
dir_save = '/home/rich/Desktop/'
filename_prefix = 'mouse_1'

##### Option 1: Save as pickled dictionary

In [15]:
filepath_noSuffix = str(Path(dir_save) / (filename_prefix + '.ROICaT.classification_predictions'))

roicat.helpers.pickle_save(
    obj=results,
    filepath=filepath_noSuffix + '.pkl',
)

##### Option 2: Save as pickled numpy array

In [17]:
np.save(
    file=filepath_noSuffix + '.npy',
    arr=results,
    allow_pickle=True
)